In [ ]:
#| default_exp core
# TODO: Change 'core' to your target module name (e.g., utils, base, models, etc.)

# Core Module

> Foundation classes and functions for {PROJECT_NAME}

This notebook demonstrates the core functionality of the library. It serves as both:
- **Documentation**: Explains what the code does and why
- **Implementation**: Contains the actual exported Python code
- **Tests**: Validates behavior with executable examples

## Setup

Import dependencies needed for both the library and notebook execution.

In [ ]:
#| export
# Core library imports - these will be exported to the Python module
from typing import Optional, List, Dict, Any, Union
from dataclasses import dataclass
from pathlib import Path
import logging

In [ ]:
#| hide
# Notebook-only imports - use tag-core test utilities instead of fastcore.test directly
from tag.core.utils.my_test import *
from fastcore.utils import *

In [ ]:
#| export
# Configure logging for the module
logger = logging.getLogger(__name__)

In [ ]:
#| hide
# Notebook-only setup
%load_ext autoreload
%autoreload 2

## Core Data Structures

Define the fundamental data structures used throughout the library.

In [ ]:
#| export
@dataclass
class DataModel:
    """
    Core data model representing {DESCRIPTION}.
    
    This class demonstrates:
    - Using dataclasses for clean data modeling
    - Type hints for clarity
    - Docstrings following Google/NumPy style
    
    Args:
        name: Human-readable identifier
        value: Numeric value associated with this item
        metadata: Optional dictionary for additional data
    
    Example:
        ```python
        item = DataModel(name="example", value=42)
        print(item.name)  # "example"
        ```
    """
    name: str
    value: float
    metadata: Optional[Dict[str, Any]] = None
    
    def __post_init__(self):
        """Validate data after initialization."""
        if not self.name:
            raise ValueError("name cannot be empty")
        if self.metadata is None:
            self.metadata = {}
    
    def is_positive(self) -> bool:
        """Check if value is positive."""
        return self.value > 0

### Testing DataModel

Demonstrate usage and validate behavior with tests using tag-core test utilities.

In [ ]:
# Create instance
item = DataModel(name="test", value=42.0)
print(f"Created: {item}")

# Test basic functionality using tag.core.utils.my_test
test_eq(item.name, "test")
test_eq(item.value, 42.0)
test_eq(item.is_positive(), True)
test_eq(item.metadata, {})

# Test with metadata
item2 = DataModel(name="example", value=-10.0, metadata={"tag": "negative"})
test_eq(item2.is_positive(), False)
test_eq(item2.metadata["tag"], "negative")

# Test validation
test_fail(lambda: DataModel(name="", value=0), contains="cannot be empty")

## Core Processing Functions

Implement the main processing logic for the library.

In [ ]:
#| export
def process_data(
    items: List[DataModel],
    threshold: float = 0.0,
    reverse: bool = False
) -> List[DataModel]:
    """
    Process a collection of data items with filtering and sorting.
    
    This function demonstrates:
    - Working with collections of custom objects
    - Configurable behavior via parameters
    - Clear documentation of processing logic
    
    Args:
        items: Collection of DataModel instances to process
        threshold: Minimum value to include (default: 0.0)
        reverse: Sort in descending order if True (default: False)
    
    Returns:
        Filtered and sorted list of DataModel instances
    
    Example:
        ```python
        items = [
            DataModel("a", 10.0),
            DataModel("b", -5.0),
            DataModel("c", 20.0)
        ]
        result = process_data(items, threshold=0.0)
        # Returns [DataModel("a", 10.0), DataModel("c", 20.0)]
        ```
    """
    # Filter based on threshold
    filtered = [item for item in items if item.value >= threshold]
    
    # Sort by value
    sorted_items = sorted(filtered, key=lambda x: x.value, reverse=reverse)
    
    logger.debug(f"Processed {len(items)} items -> {len(sorted_items)} results")
    
    return sorted_items

### Testing process_data

Validate with various scenarios.

In [ ]:
# Create test data
test_items = [
    DataModel("first", 10.0),
    DataModel("second", -5.0),
    DataModel("third", 20.0),
    DataModel("fourth", 0.0)
]

# Test basic filtering
result = process_data(test_items, threshold=0.0)
test_eq(len(result), 3)  # Excludes negative value
test_eq([r.name for r in result], ["fourth", "first", "third"])

# Test with higher threshold
result = process_data(test_items, threshold=5.0)
test_eq(len(result), 2)
test_eq([r.name for r in result], ["first", "third"])

# Test reverse sorting
result = process_data(test_items, threshold=0.0, reverse=True)
test_eq([r.name for r in result], ["third", "first", "fourth"])

# Test empty list
result = process_data([])
test_eq(result, [])

print("✓ All tests passed!")

## Advanced: Processor Class

For more complex scenarios, use a class to maintain state and configuration.

In [ ]:
#| export
class DataProcessor:
    """
    Stateful processor for DataModel collections.
    
    This class demonstrates:
    - Object-oriented design patterns
    - Configuration management
    - State tracking
    
    Args:
        threshold: Default threshold for filtering
        auto_sort: Enable automatic sorting (default: True)
    
    Example:
        ```python
        processor = DataProcessor(threshold=5.0)
        items = [DataModel("a", 10), DataModel("b", 3)]
        result = processor.process(items)
        print(processor.stats)  # {'processed': 2, 'filtered': 1}
        ```
    """
    
    def __init__(self, threshold: float = 0.0, auto_sort: bool = True):
        self.threshold = threshold
        self.auto_sort = auto_sort
        self.stats = {'processed': 0, 'filtered': 0}
    
    def process(
        self, 
        items: List[DataModel], 
        threshold: Optional[float] = None
    ) -> List[DataModel]:
        """
        Process items using instance configuration.
        
        Args:
            items: Items to process
            threshold: Override instance threshold if provided
        
        Returns:
            Processed items
        """
        # Use instance threshold if not overridden
        thresh = threshold if threshold is not None else self.threshold
        
        # Update statistics
        self.stats['processed'] += len(items)
        
        # Filter
        result = [item for item in items if item.value >= thresh]
        self.stats['filtered'] += len(items) - len(result)
        
        # Sort if enabled
        if self.auto_sort:
            result = sorted(result, key=lambda x: x.value)
        
        return result
    
    def reset_stats(self):
        """Reset processing statistics."""
        self.stats = {'processed': 0, 'filtered': 0}

### Testing DataProcessor

In [ ]:
# Create processor with default threshold
processor = DataProcessor(threshold=5.0)

# Process test data
items = [
    DataModel("a", 10.0),
    DataModel("b", 3.0),
    DataModel("c", 15.0)
]

result = processor.process(items)
test_eq(len(result), 2)  # Only items >= 5.0
test_eq([r.name for r in result], ["a", "c"])  # Auto-sorted

# Check statistics
test_eq(processor.stats['processed'], 3)
test_eq(processor.stats['filtered'], 1)

# Test threshold override
result = processor.process(items, threshold=0.0)
test_eq(len(result), 3)  # All items included

# Test without auto-sort
processor2 = DataProcessor(threshold=0.0, auto_sort=False)
result = processor2.process(items)
test_eq([r.name for r in result], ["a", "b", "c"])  # Original order

# Test reset
processor.reset_stats()
test_eq(processor.stats, {'processed': 0, 'filtered': 0})

print("✓ All processor tests passed!")

## Utility Functions

Helper functions that support the core functionality.

In [ ]:
#| export
def summarize_collection(items: List[DataModel]) -> Dict[str, Any]:
    """
    Generate summary statistics for a collection.
    
    Args:
        items: Collection to summarize
    
    Returns:
        Dictionary with count, min, max, mean, and positive_count
    """
    if not items:
        return {
            'count': 0,
            'min': None,
            'max': None,
            'mean': None,
            'positive_count': 0
        }
    
    values = [item.value for item in items]
    
    return {
        'count': len(items),
        'min': min(values),
        'max': max(values),
        'mean': sum(values) / len(values),
        'positive_count': sum(1 for item in items if item.is_positive())
    }

In [ ]:
# Test summary function
items = [
    DataModel("a", 10.0),
    DataModel("b", -5.0),
    DataModel("c", 20.0)
]

summary = summarize_collection(items)
test_eq(summary['count'], 3)
test_eq(summary['min'], -5.0)
test_eq(summary['max'], 20.0)
test_close(summary['mean'], 8.33, eps=0.01)  # Using test_close for floating point
test_eq(summary['positive_count'], 2)

# Test empty collection
empty_summary = summarize_collection([])
test_eq(empty_summary['count'], 0)
test_eq(empty_summary['min'], None)

print("✓ Summary tests passed!")

## Integration Example

Putting it all together in a realistic workflow.

In [ ]:
# Create a realistic dataset
dataset = [
    DataModel("item_1", 15.5, {"category": "A"}),
    DataModel("item_2", -3.2, {"category": "B"}),
    DataModel("item_3", 42.0, {"category": "A"}),
    DataModel("item_4", 8.7, {"category": "C"}),
    DataModel("item_5", -1.0, {"category": "B"}),
    DataModel("item_6", 23.4, {"category": "A"}),
]

print("Original dataset:")
print(summarize_collection(dataset))
print()

# Process with function
print("Using process_data function:")
result = process_data(dataset, threshold=10.0)
print(f"Filtered to {len(result)} items: {[r.name for r in result]}")
print()

# Process with class
print("Using DataProcessor class:")
processor = DataProcessor(threshold=0.0, auto_sort=True)
result = processor.process(dataset)
print(f"Processed {len(result)} items")
print(f"Statistics: {processor.stats}")
print(f"Summary: {summarize_collection(result)}")

## Best Practices Demonstrated

This notebook illustrates several nbdev and Python best practices:

1. **Clear Structure**: Logical progression from imports → data structures → functions → classes → utilities
2. **Documentation**: Every exported item has detailed docstrings
3. **Type Hints**: All functions and methods specify parameter and return types
4. **Testing**: Tests accompany every code section using `tag.core.utils.my_test`
5. **Examples**: Concrete usage examples in docstrings and test cells
6. **Separation**: `#|export` clearly marks library code vs. notebook-only code
7. **Incremental**: Building from simple to complex (dataclass → function → class)
8. **Custom Test Utils**: Uses project-specific test utilities (`tag.core.utils.my_test`) that extend fastcore

## Customization Guide

To adapt this template for your project:

1. **Update `#|default_exp`**: Change `core` to your module name
2. **Replace placeholders**: Search for `{PROJECT_NAME}`, `{DESCRIPTION}`, etc.
3. **Modify DataModel**: Replace with your actual data structures
4. **Update functions**: Replace `process_data` with your core logic
5. **Adapt processor**: Modify `DataProcessor` for your use case
6. **Add imports**: Include your specific dependencies
7. **Update tests**: Create tests specific to your functionality
8. **Test imports**: Ensure `tag.core.utils.my_test` is available (install tag-core as dependency)
9. **Expand sections**: Add new sections as needed (e.g., I/O, configuration, etc.)

## Export

This cell exports the notebook to a Python module. Run `nbdev_export` in the terminal or let git hooks handle it automatically.

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()